In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
from bs4 import BeautifulSoup as bs
import re
import pickle

In [2]:
# Found using Postman and intercepting traffics while visiting topuniversities.com
top_universities_url = 'https://www.topuniversities.com'
top_universities_ranking_url = top_universities_url + '/sites/default/files/qs-rankings-data/357051.txt'

the_universities_url = 'https://www.timeshighereducation.com'
the_universities_ranking_url = the_universities_url + '/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'

## TopUniversities.com

In [3]:
# Remove any characters that is not part of a floating number
def match_number(text):
    ''' [-+]?[0-9]*\.?[0-9]+ '''
    return float(re.sub('[^-+0-9.]', '', text))

# Extract a float number from an html document using the selector
def extract_number(html, selector):
    number_tag = html.select_one(selector)
    if number_tag is None:
        return float('NaN')
    else:
        return match_number(number_tag.get_text())

def fetch_top_university_infos(university):
    # Fetch the Webpage of the university
    university_request = rq.get(top_universities_url + university['url'])
    university_html = bs(university_request.text, 'html.parser')
    university_data = dict()
    
    # Scrap data from the JSON and the Webpage
    university_data['name'] = university['title']
    university_data['rank'] = int(match_number(university['rank_display']))
    university_data['region'] = university['region']
    university_data['country'] = university['country']
    university_data['faculty_total'] = extract_number(university_html, 'div.faculty.total div.number')
    university_data['faculty_inter'] = extract_number(university_html, 'div.faculty.inter div.number')
    university_data['faculty_local'] = university_data['faculty_total'] - university_data['faculty_inter']
    university_data['student_total'] = extract_number(university_html, 'div.student.total div.number')
    university_data['student_inter'] = extract_number(university_html, 'div.inter.total div.number')
    university_data['student_local'] = university_data['student_total'] - university_data['student_inter']
    
    return university_data

try:
    # Try to load the serialized data
    top_universities_file = open('top_universities.pickle', 'rb')
    top_universities = pickle.load(top_universities_file)
except:
    # If loading from the serialized data failed, scrap from the web!
    top_universities_json = (rq.get(top_universities_ranking_url).json())['data']
    top_universities = pd.DataFrame(list(map(fetch_top_university_infos, top_universities_json[0:200])))
    
    top_universities.region = top_universities.region.astype('category')
    top_universities.country = top_universities.country.astype('category')
    
    # Serialize the result for future runs
    top_universities_file = open('top_universities.pickle', 'wb')
    pickle.dump(top_universities, top_universities_file)
    
# Create the mapping country => region for the Times scrapper
country_regions = dict()
for country_region_pair, universities in top_universities.groupby(['country', 'region']):
    country_regions[country_region_pair[0]] = country_region_pair[1]
    
# Manually add missing data for the Times ranking
country_regions['Luxembourg'] = 'Europe'
country_regions['Russian Federation'] = 'Europe'
    
top_universities

,country,faculty_inter,faculty_local,faculty_total,name,rank,region,student_inter,student_local,student_total
0,United States,1679.0,1303.0,2982.0,Massachusetts Institute of Technology (MIT),1,North America,3717.0,7350.0,11067.0
1,United States,2042.0,2243.0,4285.0,Stanford University,2,North America,3611.0,12267.0,15878.0
2,United States,1311.0,3039.0,4350.0,Harvard University,3,North America,5266.0,17163.0,22429.0
3,United States,350.0,603.0,953.0,California Institute of Technology (Caltech),4,North America,647.0,1608.0,2255.0
4,United Kingdom,2278.0,3212.0,5490.0,University of Cambridge,5,Europe,6699.0,12071.0,18770.0
5,United Kingdom,2964.0,3786.0,6750.0,University of Oxford,6,Europe,7353.0,12367.0,19720.0
6,United Kingdom,2554.0,3791.0,6345.0,UCL (University College London),7,Europe,14854.0,16226.0,31080.0
7,United Kingdom,2071.0,1859.0,3930.0,Imperial College London,8,Europe,8746.0,7344.0,16090.0
8,United States,635.0,1814.0,2449.0,University of Chicago,9,North America,3379.0,10178.0,13557.0
9,Switzerland,1886.0,591.0,2477.0,ETH Zurich - Swiss Federal Institute of Techno...,10,Europe,7563.0,12252.0,19815.0


In [4]:
# Sort the universities by faculty/student ratio and international students/local students ratio
def ratio_stats(universities):
    universities['faculty_student_ratio'] = universities.faculty_total / universities.student_total
    universities['inter_student_ratio'] = universities.student_inter / universities.student_local
    
    top_by_faculty = (universities.sort_values('faculty_student_ratio', ascending = False))[:20]
    top_by_inter = (universities.sort_values('inter_student_ratio', ascending = False))[:20]
    
    return (top_by_faculty, top_by_inter)

In [5]:
# Top universities, using ratios
top_universities_by_faculty, top_universities_by_inter = ratio_stats(top_universities)

# Top universities by country, using ratios
top_country = top_universities.groupby('country').sum()
top_country_by_faculty, top_country_by_inter = ratio_stats(top_country)

# Top universities by region, using ratios
top_regions = top_universities.groupby('region').sum()
top_regions_by_faculty, top_regions_by_inter = ratio_stats(top_regions)

## Times Higher Education

In [6]:
import math

def fetch_the_university_infos(university):
    # No need to fetch more data
    university_data = dict()
    
    student_faculty_ratio = match_number(university['stats_student_staff_ratio'])
    inter_student_ratio = match_number(university['stats_pc_intl_students']) / 100.0
    
    # Scrap data from the JSON, some computation required
    university_data['name'] = university['name']
    university_data['rank'] = int(match_number(university['rank']))
    university_data['region'] = country_regions[university['location']]
    university_data['country'] = university['location']
    university_data['student_total'] = int(match_number(university['stats_number_students']))
    university_data['student_inter'] = int(university_data['student_total'] * inter_student_ratio)
    university_data['student_local'] = university_data['student_total'] - university_data['student_inter']
    university_data['faculty_total'] = int(university_data['student_total'] / student_faculty_ratio)
    university_data['faculty_inter'] = float('NaN')
    university_data['faculty_local'] = float('NaN')
    
    return university_data
    
try:
    # Try to load the serialized data
    the_universities_file = open('the_universities.pickle', 'rb')
    the_universities = pickle.load(the_universities_file)
except:
    # If loading from the serialized data failed, scrap from the web!
    the_universities_json = (rq.get(the_universities_ranking_url).json())['data']
    the_universities = pd.DataFrame(list(map(fetch_the_university_infos, the_universities_json[0:200])))
        
    the_universities.region = the_universities.region.astype('category')
    the_universities.country = the_universities.country.astype('category')
    
    # Serialize the result for future runs
    the_universities_file = open('the_universities.pickle', 'wb')
    pickle.dump(the_universities, the_universities_file)

In [7]:
# Top universities, using ratios
the_universities_by_faculty, the_universities_by_inter = ratio_stats(the_universities)

# Top universities by country, using ratios
the_country = the_universities.groupby('country').sum()
the_country_by_faculty, the_country_by_inter = ratio_stats(the_country)

# Top universities by region, using ratios
the_regions = the_universities.groupby('region').sum()
the_regions_by_faculty, the_regions_by_inter = ratio_stats(the_regions)

# Merging datasets

In [8]:
from difflib import SequenceMatcher
from collections import defaultdict
from operator import itemgetter

def proximity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# We will compute the proximity of the name between all universities of the rankings (cross product)
proximities = dict()
for top_index, top_uni in top_universities.iterrows():
    for the_index, the_uni in the_universities.iterrows():
        # Don't need to work on universities in different countries
        if top_uni['country'] != the_uni['country']:
            continue
        
        prox = proximity(top_uni['name'], the_uni['name'])
        
        # Only keep good proximity to reduce work in the next phase
        if prox >= .5:
            proximities[(top_index, the_index)] = prox

In [9]:
# Prepare universities to be matched
top_universities['matched'] = False
the_universities['matched'] = False
the_universities['merge_name'] = None
the_universities['faculty_inter'] = None
the_universities['faculty_local'] = None
the_universities['prox'] = 0.0
 

# Sort match by quality, we want the best matches to be processed first in case of conflicts
best_matches = sorted(proximities.items(), key = itemgetter(1), reverse = True)

for indexes, proximity in best_matches:
    # Get a view on both unis
    top_uni, the_uni = top_universities.loc[indexes[0]], the_universities.loc[indexes[1]]
    
    # Do the match only if both are still free
    if not top_uni['matched'] and not the_uni['matched']:
        # Mark the universities as matched
        top_universities.loc[indexes[0],'matched'] = True
        the_universities.loc[indexes[1],'matched'] = True
        # Give the same name for the merge
        the_universities.loc[indexes[1],'prox'] = proximity
        the_universities.loc[indexes[1],'merge_name'] = top_uni['name']

# Use a join to merge the datasets using the name
merged_universities = top_universities.merge(the_universities[['faculty_inter', 'faculty_local', 'faculty_total',
                                                               'student_inter', 'student_local', 'student_total',
                                                               'rank', 'inter_student_ratio','merge_name']],
                                             suffixes=('_top','_the'), how='inner', 
                                             left_on='name', right_on='merge_name')

# Drop unused columns and (the only) row where we don't have faculty numbers
merged_universities.drop(['merge_name', 'matched'], axis=1, inplace=True)
merged_universities.drop(merged_universities[merged_universities['faculty_total_top'].isnull()].index, inplace=True)

# Infer these columns for the Times as they are missing from the original data
# We infer that the ratio between the local and inter is the same as in the Top ranking
top_local_total_ratio = merged_universities['faculty_local_top']/merged_universities['faculty_total_top']
merged_universities['faculty_local_the'] = (top_local_total_ratio * merged_universities['faculty_total_the']).astype(int)
merged_universities['faculty_inter_the'] = merged_universities['faculty_total_the'] - merged_universities['faculty_local_the']

In [10]:
### Add some useful columns

# International : Local ratio
merged_universities['inter_ratio'] = (merged_universities['faculty_inter_top'] + merged_universities['student_inter_top'])/(merged_universities['faculty_local_top'] + merged_universities['student_local_top'])
# Students : Faculty ratio
merged_universities['student_faculty_ratio'] = 1/merged_universities['faculty_student_ratio']
# Percentage of international faculty members in the faculty
merged_universities['faculty_inter_percent'] = merged_universities['faculty_inter_top']/merged_universities['faculty_total_top']
# Our own ranking based on the arithmetic average of the two rankings (with a small delta proportional to the difference in ranking, in order to break ties favoring the more consistently ranked)
merged_universities['arithmetic_ranking'] = (merged_universities['rank_top'] + merged_universities['rank_the'] + abs(merged_universities['rank_top'] - merged_universities['rank_the'])/200 ).rank().astype(int)

## Correlation matrix

(The negative correlation between the rank(s) and the different columns is due to having the best rank at 1 while the "worst" number in the columns is 0.)

In the correlation matrix below, we can see that:
 - **Corr. ~0.9**: the number of local faculty members (*faculty_local_xxx*) is a strong indicator of the size of the faculty in its whole (*faculty_total_xxx*) 
 - **Corr. ~0.96**: same for the number of local students (*student_local_xxx*) and the total of students (*student_total_xxx*) 

In [11]:
def colored_matrix(matrix, first_column=None):
    if first_column:
        columns = list(matrix.columns.values)
        columns.remove(first_column)
        columns.insert(0, first_column)
        matrix = matrix[columns]
        
    return matrix.style.bar(align='zero', color=['#d65f5f', '#5fba7d'])

# Correlation matrix with some colored bars for better visual
cols = sorted(merged_universities.columns.values)
corr_matrix = merged_universities[cols].corr()
colored_matrix(corr_matrix)

,arithmetic_ranking,faculty_inter_percent,faculty_inter_the,faculty_inter_top,faculty_local_the,faculty_local_top,faculty_student_ratio,faculty_total_the,faculty_total_top,inter_ratio,inter_student_ratio_the,inter_student_ratio_top,rank_the,rank_top,student_faculty_ratio,student_inter_the,student_inter_top,student_local_the,student_local_top,student_total_the,student_total_top
arithmetic_ranking,1,-0.242642,-0.384923,-0.483983,-0.204382,-0.293129,-0.393827,-0.294482,-0.437561,-0.286407,-0.226129,-0.263681,0.919419,0.923912,0.274666,-0.289975,-0.290732,-0.0447655,-0.0468562,-0.110985,-0.121043
faculty_inter_percent,-0.242642,1,0.457341,0.596793,-0.481856,-0.433853,0.0861958,-0.251286,-0.0812833,0.667532,0.562196,0.583881,-0.113745,-0.345941,-0.117843,0.32482,0.315832,-0.316498,-0.332535,-0.20126,-0.194598
faculty_inter_the,-0.384923,0.457341,1,0.813299,0.383445,0.325145,0.226055,0.642663,0.604585,0.21998,0.1322,0.173872,-0.255392,-0.45605,-0.215415,0.418065,0.441369,0.254504,0.225023,0.328699,0.314633
faculty_inter_top,-0.483983,0.596793,0.813299,1,0.0726454,0.303877,0.325637,0.324238,0.668505,0.357944,0.246833,0.300181,-0.376363,-0.526591,-0.302955,0.544602,0.580932,0.179339,0.151759,0.293037,0.291303
faculty_local_the,-0.204382,-0.481856,0.383445,0.0726454,1,0.798645,0.107717,0.954013,0.654771,-0.362456,-0.343666,-0.3303,-0.212744,-0.169211,-0.0769205,0.0518746,0.0727145,0.583631,0.580431,0.531011,0.51393
faculty_local_top,-0.293129,-0.433853,0.325145,0.303877,0.798645,1,0.230695,0.768055,0.911682,-0.310611,-0.309499,-0.28102,-0.332289,-0.214501,-0.218185,0.202704,0.234805,0.654631,0.645773,0.631111,0.614768
faculty_student_ratio,-0.393827,0.0861958,0.226055,0.325637,0.107717,0.230695,1,0.162729,0.320531,0.117127,0.106738,0.0900812,-0.324096,-0.400824,-0.828411,-0.225283,-0.245365,-0.39459,-0.422833,-0.40574,-0.42812
faculty_total_the,-0.294482,-0.251286,0.642663,0.324238,0.954013,0.768055,0.162729,1,0.739402,-0.229278,-0.242182,-0.217568,-0.259377,-0.288391,-0.133728,0.178726,0.203577,0.566761,0.554537,0.547191,0.528456
faculty_total_top,-0.437561,-0.0812833,0.604585,0.668505,0.654771,0.911682,0.320531,0.739402,1,-0.0880913,-0.135144,-0.0899045,-0.421714,-0.394559,-0.300982,0.393118,0.433845,0.588367,0.569557,0.619043,0.605538
inter_ratio,-0.286407,0.667532,0.21998,0.357944,-0.362456,-0.310611,0.117127,-0.229278,-0.0880913,1,0.957189,0.99184,-0.225117,-0.318107,-0.136528,0.43166,0.41825,-0.389963,-0.391194,-0.240251,-0.21588


### Correlations in the TopUniversities ranking

We can see that the most correlated number is the number of international faculty members followed by other faculty measures, hinting us that the faculty aspect is quite important in this ranking. We can also see that the ratio student : faculty is correlated to a worst rank.

In [12]:
colored_matrix(corr_matrix.sort_values('rank_top'), 'rank_top')

,rank_top,arithmetic_ranking,faculty_inter_percent,faculty_inter_the,faculty_inter_top,faculty_local_the,faculty_local_top,faculty_student_ratio,faculty_total_the,faculty_total_top,inter_ratio,inter_student_ratio_the,inter_student_ratio_top,rank_the,student_faculty_ratio,student_inter_the,student_inter_top,student_local_the,student_local_top,student_total_the,student_total_top
faculty_inter_top,-0.526591,-0.483983,0.596793,0.813299,1,0.0726454,0.303877,0.325637,0.324238,0.668505,0.357944,0.246833,0.300181,-0.376363,-0.302955,0.544602,0.580932,0.179339,0.151759,0.293037,0.291303
faculty_inter_the,-0.45605,-0.384923,0.457341,1,0.813299,0.383445,0.325145,0.226055,0.642663,0.604585,0.21998,0.1322,0.173872,-0.255392,-0.215415,0.418065,0.441369,0.254504,0.225023,0.328699,0.314633
faculty_student_ratio,-0.400824,-0.393827,0.0861958,0.226055,0.325637,0.107717,0.230695,1,0.162729,0.320531,0.117127,0.106738,0.0900812,-0.324096,-0.828411,-0.225283,-0.245365,-0.39459,-0.422833,-0.40574,-0.42812
faculty_total_top,-0.394559,-0.437561,-0.0812833,0.604585,0.668505,0.654771,0.911682,0.320531,0.739402,1,-0.0880913,-0.135144,-0.0899045,-0.421714,-0.300982,0.393118,0.433845,0.588367,0.569557,0.619043,0.605538
faculty_inter_percent,-0.345941,-0.242642,1,0.457341,0.596793,-0.481856,-0.433853,0.0861958,-0.251286,-0.0812833,0.667532,0.562196,0.583881,-0.113745,-0.117843,0.32482,0.315832,-0.316498,-0.332535,-0.20126,-0.194598
inter_ratio,-0.318107,-0.286407,0.667532,0.21998,0.357944,-0.362456,-0.310611,0.117127,-0.229278,-0.0880913,1,0.957189,0.99184,-0.225117,-0.136528,0.43166,0.41825,-0.389963,-0.391194,-0.240251,-0.21588
faculty_total_the,-0.288391,-0.294482,-0.251286,0.642663,0.324238,0.954013,0.768055,0.162729,1,0.739402,-0.229278,-0.242182,-0.217568,-0.259377,-0.133728,0.178726,0.203577,0.566761,0.554537,0.547191,0.528456
inter_student_ratio_top,-0.283793,-0.263681,0.583881,0.173872,0.300181,-0.3303,-0.28102,0.0900812,-0.217568,-0.0899045,0.99184,0.964753,1,-0.217761,-0.111883,0.443187,0.430379,-0.375267,-0.375077,-0.224369,-0.198787
student_inter_top,-0.272494,-0.290732,0.315832,0.441369,0.580932,0.0727145,0.234805,-0.245365,0.203577,0.433845,0.41825,0.322226,0.430379,-0.273611,0.278764,0.914414,1,0.404124,0.418413,0.583492,0.635112
student_inter_the,-0.271235,-0.289975,0.32482,0.418065,0.544602,0.0518746,0.202704,-0.225283,0.178726,0.393118,0.43166,0.417861,0.443187,-0.277022,0.264553,1,0.914414,0.346338,0.364086,0.553202,0.565008


### Correlations in the Times ranking

We can see that the best indicators for the Times ranking are related to the number of the faculty members but of the TopUniversity rankings (!). Looking only at the Times own numbers, we can see that we have less correlations with these than in the TopUniversities ranking, with the most correlated number being the ratio of international students. The lack of clear correlation leads us to believe that the ranking takes numbers that weren't published (or analysed here) into account.

In [13]:
colored_matrix(corr_matrix.sort_values('rank_the'), 'rank_the')

,rank_the,arithmetic_ranking,faculty_inter_percent,faculty_inter_the,faculty_inter_top,faculty_local_the,faculty_local_top,faculty_student_ratio,faculty_total_the,faculty_total_top,inter_ratio,inter_student_ratio_the,inter_student_ratio_top,rank_top,student_faculty_ratio,student_inter_the,student_inter_top,student_local_the,student_local_top,student_total_the,student_total_top
faculty_total_top,-0.421714,-0.437561,-0.0812833,0.604585,0.668505,0.654771,0.911682,0.320531,0.739402,1,-0.0880913,-0.135144,-0.0899045,-0.394559,-0.300982,0.393118,0.433845,0.588367,0.569557,0.619043,0.605538
faculty_inter_top,-0.376363,-0.483983,0.596793,0.813299,1,0.0726454,0.303877,0.325637,0.324238,0.668505,0.357944,0.246833,0.300181,-0.526591,-0.302955,0.544602,0.580932,0.179339,0.151759,0.293037,0.291303
faculty_local_top,-0.332289,-0.293129,-0.433853,0.325145,0.303877,0.798645,1,0.230695,0.768055,0.911682,-0.310611,-0.309499,-0.28102,-0.214501,-0.218185,0.202704,0.234805,0.654631,0.645773,0.631111,0.614768
faculty_student_ratio,-0.324096,-0.393827,0.0861958,0.226055,0.325637,0.107717,0.230695,1,0.162729,0.320531,0.117127,0.106738,0.0900812,-0.400824,-0.828411,-0.225283,-0.245365,-0.39459,-0.422833,-0.40574,-0.42812
student_inter_the,-0.277022,-0.289975,0.32482,0.418065,0.544602,0.0518746,0.202704,-0.225283,0.178726,0.393118,0.43166,0.417861,0.443187,-0.271235,0.264553,1,0.914414,0.346338,0.364086,0.553202,0.565008
student_inter_top,-0.273611,-0.290732,0.315832,0.441369,0.580932,0.0727145,0.234805,-0.245365,0.203577,0.433845,0.41825,0.322226,0.430379,-0.272494,0.278764,0.914414,1,0.404124,0.418413,0.583492,0.635112
faculty_total_the,-0.259377,-0.294482,-0.251286,0.642663,0.324238,0.954013,0.768055,0.162729,1,0.739402,-0.229278,-0.242182,-0.217568,-0.288391,-0.133728,0.178726,0.203577,0.566761,0.554537,0.547191,0.528456
faculty_inter_the,-0.255392,-0.384923,0.457341,1,0.813299,0.383445,0.325145,0.226055,0.642663,0.604585,0.21998,0.1322,0.173872,-0.45605,-0.215415,0.418065,0.441369,0.254504,0.225023,0.328699,0.314633
inter_ratio,-0.225117,-0.286407,0.667532,0.21998,0.357944,-0.362456,-0.310611,0.117127,-0.229278,-0.0880913,1,0.957189,0.99184,-0.318107,-0.136528,0.43166,0.41825,-0.389963,-0.391194,-0.240251,-0.21588
inter_student_ratio_top,-0.217761,-0.263681,0.583881,0.173872,0.300181,-0.3303,-0.28102,0.0900812,-0.217568,-0.0899045,0.99184,0.964753,1,-0.283793,-0.111883,0.443187,0.430379,-0.375267,-0.375077,-0.224369,-0.198787


## Best university is... Stanford University

Using an arithmetic average between the two rankings for each university and adding a tie breaker delta that favors the university with the smallest difference between its two rankings, we obtain that Stanford is the best university with its 2nd and 3rd places in the TopUnivesities and Times rankings.

In [14]:
merged_universities.sort_values('arithmetic_ranking')[['name','rank_top', 'rank_the', 'arithmetic_ranking']].head()

,name,rank_top,rank_the,arithmetic_ranking
1,Stanford University,2,3,1
0,Massachusetts Institute of Technology (MIT),1,5,2
3,California Institute of Technology (Caltech),4,3,3
4,University of Cambridge,5,2,4
5,University of Oxford,6,1,5
